# Exploration.

In [172]:
from pymongo import MongoClient
import pandas as pd
import time
import importlib
import geopandas as gpd

import json
from cartoframes.viz import Map, Layer, popup_element

from getpass import getpass
import os
import requests

from dotenv import load_dotenv
import pymongo
import warnings
warnings.filterwarnings("ignore")
import data.exploration as ex
importlib.reload(ex)

<module 'data.exploration' from 'd:\\ironhack\\proyectos\\GeoSpatialData_proy3\\data\\exploration.py'>

## 1. Ubication.
### 1.1. Import from Mongo.

Todo el proceso para descargar la base de datos Empresas de Mongo.

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [4]:
db = client["Ironhack"]

collection for companies Mongo.

In [5]:
c = db.get_collection("companies")

collection for NY neighberhood.

In [6]:
nyneights = db.get_collection("nyneigh")

Explore the data base to get the information I want for the company.

In [7]:
# This was to look if the data base from Mongo was well downloaded.
    # c.find_one()
# This was to find with a specific condition of the ategory, and look for what i wanted to work on.
    # list(c.find({}, {"category_code":1}))

#### Will choose the music industry for my company.
Getting the music buissnes companies and start analizing them.

In [8]:
# list of all the companies with category of music in all the world.
    # list(c.find({"category_code":"music"}, {"name":1, "_id":0, "category_code":1}))

### 1.2. Ubication COUNTRY.

In [9]:
# list of the countries with the music company industry.
lstcountry = ex.music_countries ("music", 1)

In [10]:
#countries[0]
#countries[0]["offices"]
#countries[0]["offices"][0]["country_code"]

In [11]:
# change it to a dictionary
country = {"country": lstcountry}

In [12]:
# make it a data frame
dfcountry = ex.make_df (country)

We see that for much USA is the place for a music company, I wanted to see if I could go to another country that had some type of competition between the USA, but if the company wants to grow, need to be in the USA.

hacer otra columna del value count de los paises

In [13]:
dfcountry.sample(3)

,country
1,USA
23,SWE
20,LUX


### 1.3. Searching for the city

In [14]:
# Extracting from mongo list of cities chosing the category of the business.

In [15]:
lstcity = ex.music_cities ("music", 1)

Merge the 2 columns of country and city and new count column

In [16]:
location_df = ex.combine_lists (lstcity, lstcountry)

In [17]:
location_df.sample(3)

,cities,country,count
29,Framingham,USA,1
13,San Francisco,USA,9
0,San Diego,USA,2


Clean the data frame, from dulicates and nulls and create a new column for counts of offices in each city.

In [18]:
location_clean = ex.clean_df_citycountry (location_df)
location_clean.sample(3)

,cities,country,count
11,Oxford,GBR,1
0,San Diego,USA,2
28,Hamburg,DEU,1


Export the data frame of location into a CSV file.

In [19]:
ex.export_location (location_clean)

## 2. Searching for music competition.

Filtering from mongo name, latitude and longitud. to a dictionary.

In [20]:
music_ny = ex.music_NY ("music", "New York")

In [119]:
lat_music = ex.lat_ (music_ny)

In [120]:
long_music = ex.long_ (music_ny)

Create DF of latitude and longitude.

In [23]:
df_music = ex.coord_df (lat_music, long_music)

In [24]:
musicdf = ex.musicdf_NY ("music", "New York")

In [25]:
music_df = ex.concat_axis1 (musicdf, df_music)

Geo intersection.

In [121]:
musicomps_neigh = ex.dict_format_point (lat_music[:3], long_music[:3])

In [122]:
musicomps_neigh_df = ex.make_df (musicomps_neigh)

Creating df with new columns, porcentage column of value of interest.

In [123]:
musicomps_count_df = ex.new_columns (musicomps_neigh_df, "music_comp. %", 0.5)

In [26]:
ex.export_NY_music (music_df)

## 3. Startup environment.

Filter information.

In [27]:
startups = ex.startups_cooord ("New York", 1000000)

In [28]:
startups_lat = ex.lat_ (startups)

In [29]:
startups_long = ex.long_ (startups)

Filter again the startups but without the latitud and logitude and turn it into a data frame.


In [30]:
start_names = ex.startups_name ("New York", 1000000)
start_names.sample(3)

,name,category_code,acquisitions
31,Pfizer,biotech,"[{'price_amount': 195000000}, {'price_amount':..."
37,Dice Holdings,web,"[{'price_amount': 4800000}, {'price_amount': 9..."
20,LivePerson,software,"[{'price_amount': None}, {'price_amount': 3000..."


Make a dictionary with all values to create a data base.

In [31]:
#     ex.price_amount (city, amount)

In [32]:
df = ex.coord_df (startups_lat, startups_long)

In [33]:
df.sample(3)

,latitud,longitude
7,40.731118,-73.991933
38,NaN,NaN
33,NaN,NaN


Concat 2 data frames.

In [34]:
startups = ex.concat_axis1 (start_names, df)

In [35]:
startups.sample()

,name,category_code,acquisitions,latitud,longitude
38,Backstage,web,[{'price_amount': 15000000}],NaN,NaN


Geo intersection.

had to create a function to clean null values.

In [174]:
lat_lst = ex.delete_null (startups_lat)

In [175]:
lon_lst = ex.delete_null (startups_long)

In [176]:
startups_neigh = ex.dict_format_point (lat_lst, lon_lst)

In [177]:
startups_neigh_df = ex.make_df (startups_neigh)

creating data frame with new columns.

In [178]:
startups_count_df = ex.new_columns (startups_neigh_df, "startups. %", 0.7)

In [37]:
ex.export_NY_startups (startups)

## 4. Foursquare.

In [38]:
token = getpass()

### 4.1. Starbucks.

In [39]:
square_starbucks = ex.foursquare ("starbucks", 13032)

In [40]:
single = ex.single_coord_name (square_starbucks[0])

In [41]:
dict_startbucks = ex.iterate_all (square_starbucks)

In [42]:
starbucks_df = ex.make_df (dict_startbucks)


Mongo. Create a new collection and add/ insert all the rows in the data frame.

In [43]:
for index, row in starbucks_df.iterrows():
    db["ny_starbucks"].insert_one(dict(row))

In [44]:
mongo_starbucks = db.get_collection("ny_starbucks")
mongo_starbucks

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'ny_starbucks')

Geo intersection.

In [114]:
starbucks_neigh = ex.geointersection_dict (mongo_starbucks)

[{'name': 'West Village'},
 {'name': 'West Village'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'West Village'},
 {'name': 'West Village'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'West Village'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'West Village'},
 {'name': 'West Village'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'Gramercy'},
 {'name': 'Midtown-Midtown South'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'SoHo-TriBeCa-Civic Center-Little Italy'},
 {'name': 'Midtown-Midtown South'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'West Village'},
 {'name': 'West Village'},
 {'name': 'Hudson Yards-Chelsea-Flatiron-Union Square'},
 {'name': 'SoHo-TriBeCa-Civic Center-Li

In [46]:
starbucks_neigh_df = ex.make_df (starbucks_neigh)

crear nueva columna con value counts de la cantidad de starbucks en cada neightberhood.
The value .3 is the 30% that I give the importance of the starbucks for the company

In [66]:
starbuks_count_df = ex.new_columns (starbucks_neigh_df, "starbucks %", 0.3)

,name,count,starbucks %
71,West Village,18,5.4
78,Gramercy,6,1.8
92,Stuyvesant Town-Cooper Village,2,0.6
94,East Village,4,1.2
95,Midtown-Midtown South,26,7.8
97,Murray Hill-Kips Bay,6,1.8
98,SoHo-TriBeCa-Civic Center-Little Italy,10,3.0
99,Hudson Yards-Chelsea-Flatiron-Union Square,28,8.4


Export data frame to csv file.

In [52]:
ex.export_NY_starbucks (starbucks_df)

### 4.2. Schools.

In [54]:
square_schools = ex.foursquare ("schools", 12009)

In [55]:
single_schools = ex.single_coord_name (square_schools[0])

In [56]:
dict_schools = ex.iterate_all (square_schools)

In [82]:
school_df = ex.make_df (dict_schools)

Mongo collection.

In [58]:
for index, row in school_df.iterrows():
    db["ny_schools"].insert_one(dict(row))

In [60]:
mongo_schools = db.get_collection("ny_schools")
mongo_schools

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'ny_schools')

Create Geo intersection.

In [68]:
schools_neigh = ex.geointersection_dict (mongo_schools)

In [83]:
schools_neigh_df = ex.make_df (schools_neigh)

Create new column with value counts and porcentage given for schools.

In [84]:
schools_count = ex.new_columns (schools_neigh_df, "schools %", 0.4)

Export data frame to csv file.

In [ ]:
ex.export_NY_schools (school_df)

### 4.3. Music scenario.

### 4.3.1. Bars.

In [72]:
square_bars = ex.foursquare ("bars", 13003)

In [73]:
single_bars = ex.single_coord_name (square_bars[0])

In [74]:
dict_bars = ex.iterate_all (square_bars)

In [80]:
bars_df = ex.make_df (dict_bars)

Mongo collection. create a new collection and add/ insert all the rows in the data frame.

In [76]:
for index, row in bars_df.iterrows():
    db["ny_bars"].insert_one(dict(row))

In [77]:
mongo_bars = db.get_collection("ny_bars")
mongo_bars

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'ny_bars')

Create Geo Intersection.

In [78]:
bars_neigh = ex.geointersection_dict (mongo_bars)

In [85]:
bars_neigh_df = ex.make_df (bars_neigh)

In [88]:
bars_count = ex.new_columns (bars_neigh_df, "bars %", 0.2)

Export data frame to csv file.

In [ ]:
ex.export_NY_bars (school_df)

### 4.3.2. Street concerts.

In [89]:
square_concerts = ex.foursquare ("concerts", 10000)

In [90]:
single_concerts = ex.single_coord_name (square_concerts[0])

In [91]:
dict_concerts = ex.iterate_all (square_concerts)

In [99]:
concerts_df = ex.make_df (dict_concerts)

Mongo collection. Create a new collection and add/ insert all the rows in the data frame.

In [93]:
for index, row in concerts_df.iterrows():
    db["ny_concerts"].insert_one(dict(row))

In [94]:
mongo_concerts = db.get_collection("ny_concerts")
mongo_concerts

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'ny_concerts')

Geo intersection.

In [95]:
concerts_neigh = ex.geointersection_dict (mongo_concerts)

In [96]:
concerts_neigh_df = ex.make_df (concerts_neigh)

In [97]:
concerts_count = ex.new_columns (concerts_neigh_df, "concerts %", 0.3)

Export Data frame as csv file.

In [ ]:
ex.export_NY_concerts (school_df)

In [ ]:
# to create a new collection in mongo.
# db["ny_schools"].insert_one({"algo":355})